In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras
board = keras.callbacks.TensorBoard(log_dir='./logs/artsnobs', histogram_freq=0, batch_size=100, write_graph=True)

#Dynamically change learning rate based on 'val_acc' over time
#change 0.8 to 0.75?
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.8, patience=7, min_lr=0.001, verbose=1)

#Stop training when 'val_loss' stops improving after 15 epochs
e_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')

#Next 4 lines allows MASSIVE images to be loaded in and used
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
PIL.Image.MAX_IMAGE_PIXELS = None

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
# model.add(Conv2D(32, (3, 3), input_shape=(175, 175, 3)))
model.add(Conv2D(32, (3, 3), input_shape=(500, 500, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization()) #added this
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3))) #changed 32 to 64
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3))) #added this
model.add(Activation('relu')) #added this
model.add(MaxPooling2D(pool_size=(2, 2))) #added this

In [3]:
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

# model.add(Dense(64)) #added this
# model.add(Activation('sigmoid')) #added this

model.add(Dropout(0.5))
model.add(Dense(3)) #3 for art_genre_set, 5 for dataset
model.add(Activation('softmax'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])
# model.summary()
# model.load_weights('artsnobs_weights.h5') #BE AWARE IF THIS IS ENABLED

In [5]:
batch_size = 16 #change this from 16, used 100 for dataset
# train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=40, width_shift_range=0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest')

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/art_genre_set/train/', target_size = (500, 500), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
v_generator = test_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/art_genre_set/test/', target_size = (500, 500), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')

# train_generator = train_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/training_set/', target_size = (175, 175), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
# v_generator = test_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/validation_set/', target_size = (175,175), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
# # train_generator = train_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/training_set/', target_size = (150, 150), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
# # v_generator = test_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/validation_set/', target_size = (150,150), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')

Found 19011 images belonging to 3 classes.
Found 2112 images belonging to 3 classes.


In [7]:
#change 2500 to entire size of dataset (21123)
model.fit_generator(train_generator, steps_per_epoch = 2500 // batch_size, epochs = 250, validation_data =v_generator, validation_steps = 1250 // batch_size, callbacks = [board, e_stop, reduce_lr], workers = 7)
model.save_weights('artsnobs_weights.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/250
156/156 [==============================] - 33s 210ms/step - loss: 1.1012 - acc: 0.3494 - val_loss: 1.0958 - val_acc: 0.4062
Epoch 2/250
156/156 [==============================] - 32s 204ms/step - loss: 1.0853 - acc: 0.3718 - val_loss: 1.0886 - val_acc: 0.3678
Epoch 3/250
156/156 [==============================] - 52s 334ms/step - loss: 1.0749 - acc: 0.4147 - val_loss: 1.0917 - val_acc: 0.3998
Epoch 4/250
156/156 [==============================] - 28s 183ms/step - loss: 1.0716 - acc: 0.4259 - val_loss: 1.0915 - val_acc: 0.4030
Epoch 5/250
156/156 [==============================] - 32s 203ms/step - loss: 1.0541 - acc: 0.4383 - val_loss: 1.0520 - val_acc: 0.4615
Epoch 6/250
156/156 [==============================] - 30s 190ms/step - loss: 1.0563 - acc: 0.4295 - val_loss: 1.0500 - val_acc: 0.4647
Epoch 7/250
156/156 [==============================] - 28s 182ms/step - loss: 1.0373 - a

156/156 [==============================] - 26s 169ms/step - loss: 0.9218 - acc: 0.5466 - val_loss: 0.9271 - val_acc: 0.5409
Epoch 55/250
156/156 [==============================] - 26s 165ms/step - loss: 0.9063 - acc: 0.5713 - val_loss: 0.9804 - val_acc: 0.5441
Epoch 56/250
156/156 [==============================] - 27s 175ms/step - loss: 0.9160 - acc: 0.5797 - val_loss: 0.9545 - val_acc: 0.5208
Epoch 57/250
156/156 [==============================] - 34s 219ms/step - loss: 0.9183 - acc: 0.5689 - val_loss: 0.9331 - val_acc: 0.5457

Epoch 00057: ReduceLROnPlateau reducing learning rate to 0.40959997177124025.
Epoch 58/250
156/156 [==============================] - 26s 165ms/step - loss: 0.8945 - acc: 0.5673 - val_loss: 0.9310 - val_acc: 0.5521
Epoch 59/250
156/156 [==============================] - 27s 171ms/step - loss: 0.9059 - acc: 0.5753 - val_loss: 0.9227 - val_acc: 0.5433
Epoch 60/250
156/156 [==============================] - 25s 160ms/step - loss: 0.8791 - acc: 0.5921 - val_loss: 